In [1]:
import urllib3
import urllib
from pymoodle_jku.utils.login import login, relogin
import json

client = login(None)
print(client.sesskey)

FYdqkAPbiv


In [4]:
def recursive_urlencode(data):
    """URL-encode a multidimensional dictionary.
    @param data: the data to be encoded
    @returns: the url encoded data
    """

    def recursion(data, base=[]):
        pairs = []

        for key, value in data.items():
            new_base = base + [key]
            if hasattr(value, 'values'):
                pairs += recursion(value, new_base)
            else:
                new_pair = None
                if len(new_base) > 1:
                    first = urllib.parse.quote(new_base.pop(0))
                    rest = map(lambda x: urllib.parse.quote(x), new_base)
                    new_pair = '%s[%s]=%s' % (first, ']['.join(rest), urllib.parse.quote(str(value)))
                else:
                    new_pair = '%s=%s' % (urllib.parse.quote(str(key)), urllib.parse.quote(str(value)))
                pairs.append(new_pair)
        return pairs

    return '&'.join(recursion(data))

@relogin
def make_request(client_, args):
    headers = {'Content-type': 'application/x-www-form-urlencoded'}

    courses = list(client_.courses(load_pages=False))[:2]

    courseids = {}
    extradata = {}
    for index, course in enumerate(courses):
        courseids.update({str(index): course.id})

    extradata.update({'courseids': courseids})

    data = {'moodlewssettingfilter': 'true', 'moodlewssettingfileurl': 'true'}
    data.update(extradata)

    data.update({'wsfunction': 'mod_assign_get_assignments', 'wstoken': client_.sesskey})

    data = recursive_urlencode(data)

    response = client_.session.post(
        f'https://moodle.jku.at/jku/webservice/rest/server.php?moodlewsrestformat=json&wsfunction=mod_assign_get_assignments',
        data=data,
        headers=headers)
    return response

In [6]:
response = make_request(client, None)
